In [ ]:
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
datalist  = pd.read_pickle('data_list.pickle')

In [ ]:
datalist['#track'] = 0
datalist['#track'] = list(map(int,datalist['name']))
track_id = [format(i,'06') for i in datalist['#track']]
datalist['trackID'] = track_id
del datalist['name']

In [ ]:
# 133297 파일 convert 안되서 빼버림
datalist = datalist[datalist['trackID']!='133297']
datalist.iloc[2615]

In [ ]:
psdict = {}
for i,j in enumerate(datalist.index):
    y,sr = librosa.load(r'C:/UniConverter/Converted/'+datalist['trackID'].iloc[i]+'.wav',duration = 10.06)
    ps = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    psdict[j] = ps

In [ ]:
D2 = {} # Dataset2 (time stretch 0.8)

for i,j in enumerate(datalist.index):
    y, sr = librosa.load(r'C:/UniConverter/Converted/augmented1/' + datalist['trackID'].iloc[i]+'.wav', duration=10.06)
    ps = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    if ps.shape != (128, 434): continue
    D2[j] = ps

In [ ]:
D3 = {} # Dataset3 (pitch shift 2)

for i,j in enumerate(datalist.index):
    y, sr = librosa.load(r'C:/UniConverter/Converted/augmented2/' + datalist['trackID'].iloc[i]+'.wav', duration=10.06)
    ps = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    if ps.shape != (128, 434): continue
    D3[j] = ps

In [ ]:
cnt=0

remove_ls = []
for i in psdict.keys():
    if len(psdict[i][0]) != 434:
        remove_ls.append(i)
remove_ls

for rem in remove_ls:
    del psdict[rem]
len(psdict.keys())

In [ ]:
data_origin = pd.Series(psdict, name = 'mel_spec')
data_origin = pd.concat([data_origin,datalist['Genre']],axis=1).reset_index()
data_augment1 = pd.Series(D2, name = 'mel_spec')
data_augment1 = pd.concat([data_augment1,datalist['Genre']],axis=1).reset_index()
data_augment2 = pd.Series(D3, name = 'mel_spec')
data_augment2 = pd.concat([data_augment2,datalist['Genre']],axis=1).reset_index()

In [ ]:
mel = pd.Series(psdict, name = 'mel_spec')
datalist = pd.concat([datalist,mel],axis=1)
datalist = datalist.dropna()

In [ ]:
Data = pd.concat([data_origin,data_augment1,data_augment2],axis = 0)
Data = Data.dropna()
Data = Data.reset_index()
del Data['index']

In [ ]:
Data.iloc[551]

In [ ]:
import pickle
with open('datasetaugment.pickle', 'wb') as f:
    pickle.dump(Data, f)

In [ ]:
set(datalist['Genre'])

In [2]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv1D, Conv2D, Flatten, BatchNormalization, ZeroPadding2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras import regularizers
from keras.optimizers import SGD
import keras.backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras import regularizers
import random

Using TensorFlow backend.


In [4]:
dataset  = pd.read_pickle('datasetaugment.pickle')

In [5]:
gen_dict = {}
idx = 0
for gen in dataset['Genre']:
    if gen not in gen_dict.keys():
        gen_dict[gen]=idx
        idx+=1
gen_dict

{'Hip-Hop': 0,
 'Pop': 1,
 'Electronic': 2,
 'Rock': 3,
 'Country': 4,
 'Classical': 5}

In [6]:
dataset['Genre'] = [gen_dict[gen] for gen in dataset['Genre']]

In [7]:
print('Hip-Hop:   ',len(dataset[dataset['Genre']==0]))
print('Pop:       ',len(dataset[dataset['Genre']==1]))
print('Country:   ',len(dataset[dataset['Genre']==2]))
print('Rock:      ',len(dataset[dataset['Genre']==3]))
print('Electronic:',len(dataset[dataset['Genre']==4]))
print('Classical: ',len(dataset[dataset['Genre']==5]))

Hip-Hop:    2874
Pop:        3027
Country:    2235
Rock:       1802
Electronic: 2679
Classical:  1728


In [8]:
for i,j in enumerate(dataset['mel_spec']):
    if len(j[0])!=434:
         dataset = dataset.drop(i,0)

In [9]:
# dataset = pd.concat([datalist['mel_spec'],datalist['Genre']],axis=1).reset_index()
# del dataset['index']

dataset = dataset.sample(frac=1)
dataset = dataset.dropna()

#train dev test split 8:1:1
train = dataset[:11450]
dev = dataset[11450:12945]
test = dataset[12945:]

X_train, Y_train = train['mel_spec'],train['Genre']
X_dev, Y_dev = dev['mel_spec'],dev['Genre']
X_test, Y_test = test['mel_spec'],test['Genre']
# Reshape for CNN input

X_train = np.array([x.reshape(128, 434,1) for x in X_train])
X_dev = np.array([x.reshape(128, 434, 1) for x in X_dev])
X_test = np.array([x.reshape(128, 434, 1) for x in X_test])
# One-Hot encoding for classes
Y_train = np.array(keras.utils.to_categorical(Y_train, 6))
Y_dev = np.array(keras.utils.to_categorical(Y_dev, 6))
Y_test = np.array(keras.utils.to_categorical(Y_test, 6))

In [10]:
model = Sequential()
input_shape=(128, 434, 1)

model.add(Conv2D(24, (4, 4), strides=(1, 1), input_shape=input_shape))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

# model.add(Conv2D(48, (4, 4), padding="same"))
# model.add(AveragePooling2D((2, 2), strides=(2,2)))
# model.add(Activation('relu'))

model.add(Conv2D(48, (4, 4), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(6))
model.add(Activation('softmax'))

model.summary()

W1015 21:43:51.443112 139788110554880 deprecation_wrapper.py:119] From /home/lab05/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1015 21:43:51.450310 139788110554880 deprecation_wrapper.py:119] From /home/lab05/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1015 21:43:51.452836 139788110554880 deprecation_wrapper.py:119] From /home/lab05/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1015 21:43:51.480232 139788110554880 deprecation_wrapper.py:119] From /home/lab05/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W1015 21:43

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 125, 431, 24)      408       
_________________________________________________________________
average_pooling2d_1 (Average (None, 62, 215, 24)       0         
_________________________________________________________________
activation_1 (Activation)    (None, 62, 215, 24)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 215, 48)       18480     
_________________________________________________________________
average_pooling2d_2 (Average (None, 31, 107, 48)       0         
_________________________________________________________________
activation_2 (Activation)    (None, 31, 107, 48)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 159216)            0         
__________

In [11]:
epochs = 10
batch_size = 8
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])

W1015 21:43:51.854238 139788110554880 deprecation_wrapper.py:119] From /home/lab05/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1015 21:43:51.992053 139788110554880 deprecation_wrapper.py:119] From /home/lab05/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [12]:
epochs = 1
batch_size = 10
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5315187039859204864, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 872114162751304154
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13657850159161184044
 physical_device_desc: "device: XLA_GPU device"]

In [15]:
with tf.device('/gpu:0'):
    tb_hist = keras.callbacks.TensorBoard(log_dir='C:/Users/Yang Saewon/.keras/graph', histogram_freq=0, write_graph=True, write_images=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=2)
    hist = model.fit(x=X_train, y=Y_train, epochs=epochs, batch_size=batch_size, validation_data= (X_dev, Y_dev), callbacks=[early_stopping, tb_hist]) 

Train on 11450 samples, validate on 1495 samples
Epoch 1/1
 1560/11450 [===>..........................] - ETA: 23:01 - loss: 1.7678 - acc: 0.2795

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.legend(['loss','val_loss', 'acc','val_acc'])
plt.show()

score = model.evaluate(x=X_test, y=Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
Y_train

In [ ]:
# 모델 저장
model.save('music_genre_classification.h5')

In [ ]:
# 모델 불러오기
model = load_model('music_genre_classification.h5')

In [ ]:
yhat = model.predict_classes(X_test)

for i in range(len(X_test)):
    print('file_name: ' + str(test.index[i]) + ' True: ' + str(np.argmax(Y_test[i])) + ', Predict: ' + str(yhat[i]))

In [ ]:
Y_test

In [ ]:
yhat